In [1]:
import cv2
import time 
import ipywidgets as widgets 
import IPython.display as display 
import copy
from ipywidgets import Video, Image
from IPython.display import display 
from PIL import Image 
import numpy as np
import cv2
import base64
import ipywidgets as widgets
import copy
import time
import mediapipe as mp
import IPython
import os

In [2]:
url = "http://59.152.163.68:5000/stream?src=0"

In [3]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

BG_COLOR = (192, 192, 192) # gray

In [5]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.conf = 0.6

# Images
dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.print()  # or .show(), .save()

Using cache found in /home/stu2/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-5-22 Python-3.8.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12045MiB)

Fusing layers... 


Exception: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`. Cache may be out of date, try `force_reload=True` or see https://github.com/ultralytics/yolov5/issues/36 for help.

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
import multiprocessing as multi
from mp_test import pose

In [ ]:
vid = './before/Image/1.mp4'
vid2 = url
cap = cv2.VideoCapture(vid2)
wImg = widgets.Image( 
    layout = widgets.Layout(border="solid") 
) 
display(wImg)
pool = Pool(os.cpu_count())
if cap.isOpened(): 
    ret, img = cap.read()
    while ret:
        st = time.time()

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # 동영상 파일에서 캡쳐된 이미지를 이미지 파일 스트림으로 다시 인코딩을 한다. 
        results = model(img)
        
        # xy에는 인식한 각 객체, xy 좌표로 구성된 데이터 프레임 담김
        xy = results.pandas().xyxy[0]
        # person에는 인식한 객체중 'person'만 추출
        person = xy[xy['name'] == 'person']
    
        crop_image = []
        results_crop = []
        # 이미지 
        for index, row in person.iterrows():
            # 이미지에서 사람에 해당하는 부분만 크롭
            # 사진 한장(img) -> 사람 사진 여러장(crop_image[])
            crop_image.append(img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])])          
            croped = pool.map(pose,crop_image)
            
        for res in croped:
            tmpStream = cv2.imencode(".jpeg", res)[1].tobytes()
            wImg.value = tmpStream
        """
        fps = cap.get(cv2.CAP_PROP_FPS)
        str = "FPS : %0.1f" % fps        
        cv2.putText(img, str, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),3)
        """
        ret, img = cap.read()
else:
    print("not opened")
cap.release()